In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        food_prices01 = pd.read_csv(os.path.join(dirname, filename))
path = '/kaggle/input/pakistani-food-prices-updated/wfp_food_prices_pakistan.csv'
food_prices01 = pd.read_csv(path)
        
food_prices01.columns
food_prices01.info()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
food_prices01 = food_prices01[food_prices01['umid'].notna()]
food_prices01.drop('default', axis =1)

In [ ]:
food_prices01.reset_index()

In [ ]:
food_prices01.drop(['catid', 'cmid', 'mktid', 'ptid', 'sn', 'umid'], axis = 1, inplace = True)


In [ ]:
food_prices01.drop(['default'], axis = 1, inplace = True)


In [ ]:
food_prices01.reset_index(drop = True, inplace = True)
food_prices01

In [ ]:
from hijri_converter import convert

In [ ]:
food_prices01['date'] = pd.to_datetime(food_prices01['date'])

In [ ]:
food_prices01['Hijri Date'] = food_prices01['date']
food_prices01['Hijri Date'] = pd.to_datetime(food_prices01['Hijri Date'])

In [ ]:

food_prices01['Hijri Date']=food_prices01['Hijri Date'].apply(lambda x: 
                                                              convert.Gregorian.fromdate(x).to_hijri())

In [ ]:
food_prices01['Hijri Month']=food_prices01['Hijri Date'].apply(lambda x: 
                                                            x.month_name("ar"))

In [ ]:

food_prices01['Hijri Month']



In [ ]:
food_prices01.head()

In [ ]:
food_prices01.drop(food_prices01.columns[8], axis = 1, inplace= True)


In [ ]:
food_prices01['price'] = food_prices01.price.astype(float)

In [ ]:
food_prices01.describe()

In [ ]:
dt = food_prices01
dt[dt['category'] == 'cereals and tubers'][['date','price']]

In [ ]:
dt.set_index('date')
dt[dt['category'] == 'cereals and tubers'] ['price'].describe()

In [ ]:
commodities_cat_list =  dt['category'].unique().tolist()
commodities_cat_list

In [ ]:
len( dt['cmname'].unique())

In [ ]:
commodities_list = dt['cmname'].unique().tolist()

In [ ]:
commodities_list

In [ ]:
category_price = {}
for item in commodities_cat_list:
    category_price[item] = dt[dt['category'] == item] [['date', 'price']]

In [ ]:
commodities_price = {}
for item in commodities_list:
    commodities_price[item] = dt[dt['cmname'] == item] [['date', 'price']]

In [ ]:
commodities_price.keys()

In [ ]:
d =  commodities_price['Milk - Retail']
d

In [ ]:
summary  = dt[['date', 'price', 'category']]
s = summary.reset_index()
s = s.pivot_table(index = 'date', columns='category', values='price')
same_date = dt[dt['date'] == '2014-12-15']
same_date[same_date['cmname'] == 'Wheat flour - Retail']

In [ ]:
pd.options.plotting.backend = "plotly"
year14 = dt.set_index('date').loc['2014-01-15':'2020-12-15']
year14.pivot_table(index='Hijri Month', columns = 'cmname', values = 'price').plot.line(title= 'commodity overall price trend of commmodities in Hijri months')

In [ ]:
year14 = dt.set_index('date').loc['2014-01-15':'2020-12-15']
year14.pivot_table(index='Hijri Month', columns = 'category', values = 'price').plot.line(title= 'retail overall price trend of categories in Hijri month')

In [ ]:
pd.options.plotting.backend = "plotly"
summary_cities = dt[['date', 'cmname', 'category', 'mktname', 'price']]
summary_cities= summary_cities.pivot_table(index = ['date','cmname'], columns='mktname')
summary_cities['price'].reset_index()[['date','cmname','Karachi']].interpolate(method = 'polynomial'
                                                                               , order =2).pivot_table(index='date',
                                                                                columns='cmname', values='Karachi').loc['2015-01-15':].plot.line()

In [ ]:
summary_cities['price'].reset_index()[['date','cmname','Quetta']].interpolate(method = 'polynomial'
                                                            , order =2).pivot_table(index='date',
                                                            columns='cmname', 
                                                            values='Quetta').loc['2015-01-15':].plot(title='commodities price trends in Quetta')

In [ ]:
pd.options.plotting.backend = "plotly"
summary_cities = dt[['date', 'cmname', 'category', 'mktname', 'price']]
summary_cities= summary_cities.pivot_table(index = ['date','category'], columns='mktname')
summary_cities['price'].reset_index()[['date','category','Karachi']].interpolate(method = 'polynomial'
                                                                               , order =2).pivot_table(index='date',
                                                                                columns='category',
                                                                        values='Karachi').loc['2015-01-15':].plot(title='category price trends in Karachi')

In [ ]:
d = s.loc['2015':]
d =d.interpolate(method = 'polynomial', order =2)


In [ ]:
pd.options.plotting.backend = "plotly"
fig = d.plot.line(title="Price trends in overall Pakistan")
#ax.set_title(,color='black')
fig.update_yaxes(tickprefix="$")
fig.show()

In [ ]:
key = commodities_price['Wage (non-qualified labour, non-agricultural) - Retail']
key=key.set_index('date')
key = key.loc['2015':].interpolate(method = 'polynomial', order =2)
# ax1 = key.plot.line()
# # ax1.set_title("Title here!",color='black')
# # ax1.legend(bbox_to_anchor=(1.0, 1.0))
# ax1.show()

In [ ]:
cumm_sum = summary  = dt[['date', 'price', 'cmname']]
s = summary.reset_index()
s = s.pivot_table(index = 'date', columns='cmname', values='price')
s =s.loc['2015-4':].interpolate(method = 'polynomial', order =2)
s.rename(columns= {'Wage (non-qualified labour, non-agricultural) - Retail': 'Wage/Day'}, inplace= True)
s.plot(title='Commodities Prices trends in overall Pakistan')        

In [ ]:
s['overall commodity price/ unit'] = s.mean(axis=1)

In [ ]:
s.plot(title='overall commodity price/ unit in Pakistan', y='overall commodity price/ unit')